## 1.0a Taxi Data - Preprocessing

In this first notebook, the main dataframe is preprocessed. This includes:
- Loading the data
- Changing its datatypes
- Dropping trivial or null columns/rows
- Splitting the data into subsets and saving them for further work

As the original data is rather large in size, it is not included in the repository. Instead, when wanting to run this notebook, it is necessary to:
1. Download the data from https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew#column-menu where you specify via Actions->Query Data->Trip Start Timestamp // In Between // 2022 Jan 01 12:00:00 AM AND 2022 Dec 31 11:59:59 PM
2. Rename the file to "taxidata"
3. Run the code cell below and add the data to the following directory

In [3]:
import os
# this directory for the original data file
os.makedirs('./data', exist_ok=True)
# this directory to later save the prepared data
os.makedirs('./data/prepped', exist_ok=True)

In [4]:
# Standard libraries - run pip install if necessary
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

# Geospatial libraries
from h3 import h3 
import geopandas as gp
import folium
from shapely.ops import unary_union
from shapely.geometry.polygon import Polygon
## Color for map 
import branca
import branca.colormap as cm

### 1.1 Read and display datafile

In [5]:
# Data file not included in the project, needs to be downloaded individually. This step can take a few minutes due to size of the original file
df = pd.read_csv("data/taxidata.csv")

In [6]:
df.head(3)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,4404c6835b9e74e9f74d70f235200a8ce09db14a,7e179f8ef66ae99ec2d1ec89224e0b7ee5469fe5627f6d...,2022-12-31T23:45:00.000,2023-01-01T00:15:00.000,2081.0,4.42,NaN,NaN,2.0,3.0,...,0.0,20.50,Prcard,Flash Cab,42.001571,-87.695013,POINT (-87.6950125892 42.001571027),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)
1,466473fd2a196ebe92fb2983cb7e8af32e39aa1f,d1d88b89ceb6d753007b6e795e3c24f4bea905a51e9d47...,2022-12-31T23:45:00.000,2023-01-01T00:00:00.000,812.0,0.00,NaN,NaN,8.0,24.0,...,0.0,16.57,Mobile,Flash Cab,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.901207,-87.676356,POINT (-87.6763559892 41.9012069941)
2,3f5cd3f78e5cab455606a31372a95d3204b2fb3f,847cf962bd6f62040673e6c24c24940aeb2d7fdaa54677...,2022-12-31T23:45:00.000,2023-01-01T00:00:00.000,600.0,0.90,NaN,NaN,8.0,8.0,...,3.0,12.00,Credit Card,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)


In [7]:
# Data types
df.dtypes

trip_id                        object
taxi_id                        object
trip_start_timestamp           object
trip_end_timestamp             object
trip_seconds                  float64
trip_miles                    float64
pickup_census_tract           float64
dropoff_census_tract          float64
pickup_community_area         float64
dropoff_community_area        float64
fare                          float64
tips                          float64
tolls                         float64
extras                        float64
trip_total                    float64
payment_type                   object
company                        object
pickup_centroid_latitude      float64
pickup_centroid_longitude     float64
pickup_centroid_location       object
dropoff_centroid_latitude     float64
dropoff_centroid_longitude    float64
dropoff_centroid_location      object
dtype: object

In [8]:
# Convert time types to check if entries are from correct range
df["trip_start_timestamp"] = pd.to_datetime(df["trip_start_timestamp"])
df["trip_end_timestamp"] = pd.to_datetime(df["trip_end_timestamp"])

# In range of 2022:
print(f"Min date: {df['trip_start_timestamp'].min()}")
print(f"Max date: {df['trip_start_timestamp'].max()}")

# Convert other 
df["trip_seconds"] =  pd.to_numeric(df['trip_seconds'])

Min date: 2022-01-01 00:00:00
Max date: 2022-12-31 23:45:00


Make sure these are the right dates, here it should say **2022-01-01 00:00:00** and **2022-12-31 23:45:00**. 

In [9]:
# Convert trip duration to hours
df['trip_hours'] = df['trip_seconds'] / 3600

In [10]:
# Look into null values
print(f"General shape of dataframe: {df.shape}")
print(df.isna().sum())

General shape of dataframe: (6382425, 24)
trip_id                             0
taxi_id                           354
trip_start_timestamp                0
trip_end_timestamp                212
trip_seconds                     1465
trip_miles                         56
pickup_census_tract           3758594
dropoff_census_tract          3707094
pickup_community_area          513853
dropoff_community_area         633684
fare                             3536
tips                             3536
tolls                            3536
extras                           3536
trip_total                       3536
payment_type                        0
company                             0
pickup_centroid_latitude       511551
pickup_centroid_longitude      511551
pickup_centroid_location       511551
dropoff_centroid_latitude      597931
dropoff_centroid_longitude     597931
dropoff_centroid_location      597931
trip_hours                       1465
dtype: int64


In [11]:
# ------------------ Columns we do/will not work with can be dropped here ------------------

### 1.2 Checking data logic & removing invalid data

#### 1.2.1 Duplicate entries

In [12]:
# Check duplicates 
print("Number of duplicate entries: ", df.duplicated(subset = ['taxi_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds', 'trip_miles', 
                                                              'pickup_census_tract','dropoff_census_tract','pickup_community_area', 'dropoff_community_area',
                                                              'pickup_centroid_latitude','pickup_centroid_longitude','pickup_centroid_location','dropoff_centroid_latitude',
                                                              'dropoff_centroid_longitude','dropoff_centroid_location']).sum())

Number of duplicate entries:  21772


In [13]:
df = df.drop_duplicates(subset = ['taxi_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds', 'trip_miles', 
                                                              'pickup_census_tract','dropoff_census_tract','pickup_community_area', 'dropoff_community_area',
                                                              'pickup_centroid_latitude','pickup_centroid_longitude','pickup_centroid_location','dropoff_centroid_latitude',
                                                              'dropoff_centroid_longitude','dropoff_centroid_location'], keep='first')
df.shape

(6360653, 24)

#### 1.2.2 Drop Outliers

In [14]:
df_outliers = df

In [15]:
def drop_outliers(df, column, mean, std):
    return df[(df[column] > mean - 3 * std) & (df[column] < mean + 3 * std)]

In [16]:
std_trip_seconds = df_outliers['trip_seconds'].describe(include='all').loc['std']
mean_trip_seconds = df_outliers['trip_seconds'].describe(include='all').loc['mean']

std_trip_miles = df_outliers['trip_miles'].describe(include='all').loc['std']
mean_trip_miles = df_outliers['trip_miles'].describe(include='all').loc['mean']

std_trip_total = df_outliers['trip_total'].describe(include='all').loc['std']
mean_trip_total = df_outliers['trip_total'].describe(include='all').loc['mean']

In [17]:
df_outliers = drop_outliers(df_outliers, "trip_seconds", mean_trip_seconds, std_trip_seconds)
df_outliers = drop_outliers(df_outliers, "trip_miles", mean_trip_miles, std_trip_miles)
df_outliers = drop_outliers(df_outliers, "trip_total", mean_trip_total, std_trip_total)

In [18]:
df_outliers.shape

(6306811, 24)

In [21]:
# Number of equal zero entries after dropping
print("Number of zero entries for trip_seconds: ", len(df_outliers[df_outliers["trip_seconds"] == 0]))
print("Number of zero entries for trip_miles: ", len(df_outliers[df_outliers["trip_miles"] == 0]))
print("Number of zero entries for trip_total: ", len(df_outliers[df_outliers["trip_total"] == 0]))

Number of zero entries for trip_seconds:  147287
Number of zero entries for trip_miles:  781031
Number of zero entries for trip_total:  5325


#### 1.2.2 Mph Logic

To have some orientation for the speed limit in Chicago/Cook County, we took this as a source: https://www.arcgis.com/home/item.html?id=5e279cbe89794bcba87809d9ae95594d which resulted in a limit of 65 mph for our taxi data, as anything above is unrealistic.

In [22]:
df_outliers['mph'] = np.where(df_outliers['trip_hours'] != 0, df_outliers['trip_miles'] / df_outliers['trip_hours'], np.nan)

In [23]:
df_outliers = df_outliers[(df_outliers["mph"] <= 65)]

In [24]:
df_outliers["mph"].max()

65.0

In [ ]:
## change order
## duration 
## kmh

#### 1.2.2 Tips Logic

Through some research we found that the base fare of any taxi ride is $3.25 (https://www.chicago.gov/content/dam/city/depts/bacp/publicvehicleinfo/Chicabs/chicagotaxiplacard20200629.pdf). Hence, entries where the **"trip_total" is smaller than 3.25** are dropped.

In [25]:
print("Number of entries with below $3.25 trip total: ", len(df_outliers[df_outliers["trip_total"] <= 3.25]))

Number of entries with below $3.25 trip total:  99404


In [26]:
df_outliers = df_outliers[df_outliers["trip_total"] > 3.25]

In [27]:
df_outliers["trip_total"].min()

3.26

#### 1.2.3 Cancelled Trips Logic

We consider a trip cancelled if
- trip_miles = 0
- trip_seconds = 0
- pickup = dropoff (regarding lat, lng, census tract and centroid location)

In [29]:
print(len(df_outliers[df_outliers['trip_miles']==0]))

570298


In [44]:
# Filter out trips that match any "cancelled"-condition
condition_census_tract = (df_outliers['pickup_census_tract'].notna() & df_outliers['dropoff_census_tract'].notna())

condition_geolocation = (
    (df_outliers['pickup_centroid_latitude'].notna() & df_outliers['dropoff_centroid_latitude'].notna()) &
    (df_outliers['pickup_centroid_longitude'].notna() & df_outliers['dropoff_centroid_longitude'].notna())
)

condition_community_area = df_outliers['pickup_community_area'].notna() & df_outliers['dropoff_community_area'].notna()

condition = (condition_census_tract | condition_geolocation|  condition_community_area) 

df_cancelled = df_outliers[condition]

In [45]:
df_cancelled.shape

(5257291, 25)

In [46]:
df_cancelled[df_cancelled["trip_miles"]== 0].shape

(524407, 25)

For the rest of the trips, where trip_miles equals 0 but any of the pickup and dropoff data is given, we use the census data to fill in any missing entries. We use as merging points **geoid** for **"dropoff/pickup census tract"** and **commarea** for **dropoff/pickup community area**.

#### 1.2.4 Merging Census Data

In [47]:
# Load prepped census dataframe
census_df = gp.read_file("data/prepped/census_tracts_df.geojson")
census_df.head(3)

,commarea,community_name,geoid,geometry
0,44,Chatham,17031842400,"POLYGON ((-87.60518 41.72272, -87.60552 41.722..."
1,59,McKinley Park,17031840300,"POLYGON ((-87.67551 41.82310, -87.67607 41.823..."
2,34,Armour Square,17031841100,"POLYGON ((-87.62947 41.83097, -87.62938 41.830..."


In [48]:
df_cancelled[df_cancelled["pickup_census_tract"].notna()][["pickup_census_tract","dropoff_census_tract", "pickup_community_area", "dropoff_community_area" ]].head(3)

,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area
8,1.703108e+10,1.703184e+10,8.0,32.0
10,1.703132e+10,1.703132e+10,32.0,32.0
16,1.703108e+10,1.703108e+10,8.0,8.0


In [58]:
df_prepmerge = df_cancelled

In [72]:
# Change scientic notation to normal
df_prepmerge.loc[:, 'pickup_geoid'] = df_prepmerge['pickup_census_tract'].apply(lambda x: '{:.0f}'.format(x) if pd.notna(x) else np.nan)
df_prepmerge.loc[:, 'dropoff_geoid'] = df_prepmerge['dropoff_census_tract'].apply(lambda x: '{:.0f}'.format(x) if pd.notna(x) else np.nan)

In [75]:
df_prepmerge[df_prepmerge["pickup_census_tract"].notna()][['pickup_census_tract', 'dropoff_census_tract',"pickup_community_area", "dropoff_community_area", "pickup_geoid", "dropoff_geoid"]].head(3)

,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,pickup_geoid,dropoff_geoid
8,1.703108e+10,1.703184e+10,8.0,32.0,17031081700,17031839100
10,1.703132e+10,1.703132e+10,32.0,32.0,17031320100,17031320100
16,1.703108e+10,1.703108e+10,8.0,8.0,17031081700,17031081700


In [107]:
# make not null example dataframe (3 rows for testing)
test_df = df_prepmerge[df_prepmerge["pickup_census_tract"].notna()].head(3)

In [108]:
# example of merge just for when geoid given (==condition_census_tract)
def merge_on(df, df_to_merge, left_col, right_col, name):
    merged_df = pd.merge(df, df_to_merge, left_on=left_col, right_on=right_col, how='left')
    merged_df.rename(columns={
        'commarea': f'{name}_commarea',
        'community_name': f'{name}_community_name',
        'geometry': f'{name}_geometry'
    }, inplace=True)
    
    # drop the col we used to merge
    merged_df = merged_df.drop(right_col, axis=1)
    
    return merged_df

In [109]:
test_df = merge_on(test_df, census_df, "pickup_geoid", "geoid", "pickup")
test_df = merge_on(test_df, census_df, "dropoff_geoid", "geoid", "dropoff")
test_df.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,rewritten_pickup,rewritten_dropoff,pickup_geoid,dropoff_geoid,pickup_commarea,pickup_community_name,pickup_geometry,dropoff_commarea,dropoff_community_name,dropoff_geometry
0,3186be22cdd5d17def50e44eca6be9ec9b0a9974,4f78222d267c08ee7267810e5f3f2241dc61bf1396d036...,2022-12-31 23:45:00,2022-12-31 23:45:00,540.0,0.7,1.703108e+10,1.703184e+10,8.0,32.0,...,17031081700,17031839100,17031081700,17031839100,8,Near North Side,"POLYGON ((-87.62983 41.88751, -87.63073 41.887...",32,Loop,"POLYGON ((-87.60535 41.87420, -87.60533 41.873..."
1,2c7c880f01cbf322beb00a78f937b638ccee016b,44d82604888573a1d067dd0e6b1290e08eec36e9eafe32...,2022-12-31 23:45:00,2022-12-31 23:45:00,120.0,0.3,1.703132e+10,1.703132e+10,32.0,32.0,...,17031320100,17031320100,17031320100,17031320100,32,Loop,"POLYGON ((-87.60535 41.87420, -87.60533 41.873...",32,Loop,"POLYGON ((-87.60535 41.87420, -87.60533 41.873..."
2,1d104fd39db4351c83f3736b079ed37f64e7528e,f7702c24e158419dab2ab9901f181b533f0d43392b52bc...,2022-12-31 23:45:00,2022-12-31 23:45:00,15.0,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,17031081700,17031081700,17031081700,17031081700,8,Near North Side,"POLYGON ((-87.62983 41.88751, -87.63073 41.887...",8,Near North Side,"POLYGON ((-87.62983 41.88751, -87.63073 41.887..."


In [ ]:
## MERGE HERE --------------------

In [ ]:
## MERGE END -----------

In [ ]:
## if merged add rest of function

In [ ]:
def calculate_distance(row):
    if row['trip_miles'] == 0:
        if condition_geolocation:
            pickup_point = (row['pickup_centroid_latitude'], row['pickup_centroid_longitude'])
            dropoff_point = (row['dropoff_centroid_latitude'], row['dropoff_centroid_longitude'])
            return geodesic(pickup_point, dropoff_point).miles
        else if condition_census_tract:
            ##
        else if condition_community_area:
            ##
    else:
        return row['trip_miles']

### 1.3 Checking null values

In [252]:
df_cancelled.isna().sum()

trip_id                             0
taxi_id                             1
trip_start_timestamp                0
trip_end_timestamp                  0
trip_seconds                        0
trip_miles                          0
pickup_census_tract           2841436
dropoff_census_tract          2841436
pickup_community_area            1365
dropoff_community_area          33496
fare                                0
tips                                0
tolls                               0
extras                              0
trip_total                          0
payment_type                        0
company                             0
pickup_centroid_latitude          283
pickup_centroid_longitude         283
pickup_centroid_location          283
dropoff_centroid_latitude        5157
dropoff_centroid_longitude       5157
dropoff_centroid_location        5157
trip_km                             0
trip_hours                          0
mph                                 0
dtype: int64

#### 1.3.1 Time null values

In [40]:
# If start and end time are the same then trip seconds would still 

In [41]:
# Drop entries where trip seconds are null and time is equal
print("Entries where no trip seconds but start and end time are also equal ", len(df[df["trip_seconds"].isna() & (df["trip_start_timestamp"] ==  df["trip_end_timestamp"])]))
print("Entries where no trip seconds but start and end time different ", len(df[df["trip_seconds"].isna() & (df["trip_start_timestamp"] !=  df["trip_end_timestamp"])]))

Entries where no trip seconds but start and end time are also equal  0
Entries where no trip seconds but start and end time different  0


In [42]:
# Drop where cannot be calculated:
df = df[~(df["trip_seconds"].isna() & (df["trip_start_timestamp"] ==  df["trip_end_timestamp"]))]

# Else calculate the trip seconds:
temp_trip = df[df["trip_seconds"].isna() & (df["trip_start_timestamp"] !=  df["trip_end_timestamp"])].copy()
temp_trip["calculated_trip_seconds"] = (temp_trip["trip_end_timestamp"] - temp_trip["trip_start_timestamp"]).dt.seconds
df["trip_seconds"].fillna(temp_trip["calculated_trip_seconds"], inplace=True)

df["trip_seconds"].isna().sum()

0

#### Further checks...

### 1.4 Hexagons - res8 and res7 takes too long (deswegen erstmal ausgeklammert)

In [43]:
# Get hex ids
def add_h3_ids(df, res):
    df[f"h3_res{res}_pickup"] = np.vectorize(h3.geo_to_h3)(
        df['pickup_centroid_latitude'], df['pickup_centroid_longitude'], res)
    df[f"h3_res{res}_dropoff"] = np.vectorize(h3.geo_to_h3)(
        df['dropoff_centroid_latitude'], df['dropoff_centroid_longitude'], res)
    return df

# Get poly from hex ids - vectorized form to save time
def poly_from_hex(df, colname, res):
    hex_ids = df[f"h3_res{res}_{colname}"].values
    polygons = np.vectorize(lambda hex_id: Polygon(h3.h3_to_geo_boundary(hex_id, geo_json=True)))(hex_ids)
    df[f"poly_res{res}_{colname}"] = polygons
    return df

# Get count for each trip happening in the same hexagon
def get_poly_count(df, colname):
    name = colname.split("_")[1] + "_" + colname.split("_")[2]
    df[f"count{name}"] = df.groupby(colname)['trip_id'].transform('count')
    return df

In [1]:
# For hexagon resolution, adapted: https://towardsdatascience.com/exploring-location-data-using-a-hexagon-grid-3509b68b04a2 table
# hex_df = add_h3_ids(df, 7)
#hex_df = add_h3_ids(df, 8)

In [45]:
# Entries where hex id is 0, we cannot use for visualization, hence drop
## TODO: maybe if census tract is given/community area, we can use this to merge with later census data to get hexagons for these entries? 
print("Number of hex ids equal to 0: ",len(hex_df[(hex_df["h3_res7_pickup"] == "0") | (hex_df["h3_res7_dropoff"] == "0")]))

hex_df_clear = hex_df[(hex_df["h3_res7_pickup"] != "0") & (hex_df["h3_res7_dropoff"] != "0")]

Number of hex ids equal to 0:  871484


In [46]:
hex_df_clear.head(2)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,trip_km,trip_hours,Check,kmh,h3_res7_pickup,h3_res7_dropoff
0,4404c6835b9e74e9f74d70f235200a8ce09db14a,7e179f8ef66ae99ec2d1ec89224e0b7ee5469fe5627f6d...,2022-12-31 23:45:00,2023-01-01 00:15:00,2081.0,4.42,NaN,NaN,2.0,3.0,...,POINT (-87.6950125892 42.001571027),41.965812,-87.655879,POINT (-87.6558787862 41.96581197),7.1133,0.578056,True,12.305565,872664d8effffff,872664d89ffffff
1,466473fd2a196ebe92fb2983cb7e8af32e39aa1f,d1d88b89ceb6d753007b6e795e3c24f4bea905a51e9d47...,2022-12-31 23:45:00,2023-01-01 00:00:00,812.0,0.00,NaN,NaN,8.0,24.0,...,POINT (-87.6333080367 41.899602111),41.901207,-87.676356,POINT (-87.6763559892 41.9012069941),0.0000,0.225556,True,0.000000,872664c1effffff,872664cacffffff


In [47]:
# Get polygon from hex ids
#hex_df_poly = poly_from_hex(hex_df_clear, "pickup", 7)
#hex_df_poly = poly_from_hex(hex_df_clear, "dropoff", 7)
# hex_df_poly = poly_from_hex(hex_df_clear, "pickup", 8)
# hex_df_poly = poly_from_hex(hex_df_clear, "dropoff", 8)


KeyboardInterrupt



In [ ]:
hex_df_poly.head(2)

In [ ]:
# Get count for each polygon
# hex_df_poly = get_poly_count(hex_df_poly, "poly_res7_pickup")
#hex_df_poly = get_poly_count(hex_df_poly, "poly_res7_dropoff")
# hex_df_poly = get_poly_count(hex_df_poly, "poly_res8_pickup")
# hex_df_poly = get_poly_count(hex_df_poly, "poly_res8_dropoff")

In [ ]:
#hex_df_clear.head(1)

In [ ]:
# Make a geodf out of it for simple plotting
gdf_res7_pickup = gp.GeoDataFrame(hex_df_clear, geometry=hex_df_poly['poly_res7_pickup'], crs='EPSG:4326')
gdf_res7_dropoff = gp.GeoDataFrame(hex_df_clear, geometry=hex_df_poly['poly_res7_dropoff'], crs='EPSG:4326')
# gdf_res8_pickup = gp.GeoDataFrame(hex_df_clear, geometry=hex_df_poly['poly_res8_pickup'], crs='EPSG:4326')
# gdf_res8_dropoff = gp.GeoDataFrame(hex_df_clear, geometry=hex_df_poly['poly_res8_dropoff'], crs='EPSG:4326')

#Visualize
# fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize=(10, 10))

# titles = ["Hex resolution 7", 
#           # "Hex resolution 8"
#          ]
# dfs = [gdf_res7_pickup, gdf_res7_dropoff,  
#        # gdf_res8_pickup, gdf_res8_dropoff
#       ]

# axs = axs.flatten()
# for ind in range(0, 3):
#     dfs[ind].plot(column="count", ax=axs[ind], legend=True)
#     axs[ind].set_title(titles[ind])

# plt.tight_layout()
# plt.show()

In [ ]:
# df_pickup_res8['geometry'] = df_pickup_res7.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x["h3_pickup_res8"], geo_json=True)), axis=1)
# trips_starts_geo = gp.GeoDataFrame(df_pickup_res7, geometry=df_pickup_res7['geometry'], crs='EPSG:4326')
# trips_starts_geo.head()

In [ ]:
# trips_starts_geo.plot(column='count')